In [ ]:
import geopandas as gpd
from getpass import getpass
import numpy as np
from pathlib import Path
from pprint import pprint
import shutil
from typing import List, Union

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window

from rscube.nd_tools import (get_array_from_features, 
                             get_features_from_array, 
                             get_superpixel_area_as_features, 
                             get_superpixel_means_as_features,
                             get_superpixel_stds_as_features, 
                             scale_img)
from rscube.rio_tools import (get_geopandas_features_from_array, 
                              rasterize_shapes_to_array, 
                              get_indices_from_extent, get_cropped_profile)
from shapely.geometry import box
from tqdm.notebook import tqdm

from planet import api
from planet.api import downloader

This notebook downloads a Planet Scene and prepares it for the DSWx workflow.

Eventually, when HLS data becomes available, we should replace this notebook with one that downloads HLS data and prepares it in the same manner.

# 1. Download Planet Data

In [ ]:
planet_api_key = getpass("Input your Planet API key")

In [ ]:
planet_scene_id = '20211003_161639_91_241d'
# planet_scene_id = input("Enter a Planet Scene ID")

In [ ]:
def planet_download(api_key: str, scene_id: str, output_dir: Union[str, Path] = Path.cwd(), asset_types: List = ['ortho_analytic_8b_sr']):
    """
    Download Planet data by scene ID 

    Keyword arguments:
    api_key     -- Planet API key
    scene_id    -- Planet data scene ID
    output_dir  -- path to directory in which to download data
    asset_types -- Planet data asset types to download
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    client = api.ClientV1(api_key=planet_api_key)
    request = client.get_item('PSScene', 
                          scene_id)
    items_to_download = [request.get()]

    downloader = api.downloader.create(client)
    downloader.activate(iter(items_to_download), asset_types)
    downloader.download(iter(items_to_download), asset_types, str(output_dir))
    print(downloader.stats())
    downloader.shutdown()

In [ ]:
planet_dir = Path.cwd()/"planet_data"

planet_download(planet_api_key, planet_scene_id, planet_dir)

# 2. Preprocess Planet Data

### Expose metadata

In [ ]:
planet_image_path = list(planet_dir.rglob(f'{planet_scene_id}*.tif'))[0]

with rasterio.open(planet_image_path) as ds:
    planet_crs = ds.crs
    planet_box = box(*ds.bounds)
    planet_bounds = list(ds.bounds)
    planet_profile = ds.profile
    planet_shape = planet_profile['height'], planet_profile['width']

print(f'Planet Imagery Shape: {planet_shape}')
print(f'Planet Imagery CRS: {planet_crs}')
print(f'Planet Image Profile: {planet_profile}')

### Read chips geojson into a DataFrame and project to Planet data's UTM

In [ ]:
chips = gpd.read_file('ResampleblocksUpd.geojson')
chips = chips.to_crs(planet_crs)
chips.head()

### Create DataFrame of chips intersecting with Planet Data

- currently assumes a single intersecting chip, which may not always be true

In [ ]:
intersects = chips.geometry.intersects(planet_box) # create intersection mask
intersecting_chips = chips[intersects].reset_index(drop=True) # create new Dataframe containing intersecting chips
print(intersecting_chips.head())
chip_id = index = intersecting_chips.random_id[0]

### Create an image profile for the Planet Data, clipped to the chip extent 

In [ ]:
(start_y, start_x), (stop_y, stop_x) = get_indices_from_extent(planet_profile['transform'],
                                                               list(chips.total_bounds),
                                                               shape=planet_shape)
window = Window.from_slices((start_y, stop_y), (start_x, stop_x))
sx, sy = np.s_[start_x: stop_x], np.s_[start_y: stop_y]
profile_cropped = get_cropped_profile(planet_profile, sx, sy)
print(f'Cropped Planet Image Profile: {profile_cropped}')

### Remove Coastal Blue no-data pixels from all Planet Imagery bands

In [ ]:
with rasterio.open(planet_image_path) as ds:
    planet_ds = ds.read(window=window)
    planet_ds = planet_ds.transpose([1, 2, 0]).astype(np.float32)

coastal_blue_nan_mask = (planet_ds[..., 0] == planet_profile['nodata'])
planet_ds[coastal_blue_nan_mask, :] = np.nan

### Subset Planet imagery to Red, Green, NIR bands and normalize data

In [ ]:
with rasterio.open(planet_image_path) as ds:
    bands = ds.descriptions
pprint(list(enumerate(bands)))

rgnir = scale_img(planet_ds[..., [7, 5, 3]]) 

### Remove outliers (highest and lowest 2% pixel values) from all 3 bands

In [ ]:
planet_temp = planet_ds[..., [7, 5, 3]]

planet_ds_view = planet_temp.copy()
for k in tqdm(range(3)):
    m0 = np.nanpercentile(planet_temp[~coastal_blue_nan_mask, k], 2)
    m1 = np.nanpercentile(planet_temp[~coastal_blue_nan_mask, k], 98)
    planet_ds_view[~coastal_blue_nan_mask, k] = np.clip(planet_temp[~coastal_blue_nan_mask, k], m0, m1)
    
rgnir_cropped = scale_img(planet_ds_view)

### Write Planet rgnir band data to GeoTiff, with outliers removed and cropped to chip extents

In [ ]:
profile_rgnir_cropped = profile_cropped.copy()
profile_rgnir_cropped['count'] = 3
profile_rgnir_cropped['dtype'] = 'float32'
profile_rgnir_cropped['nodata'] = np.nan

preprocessed_path = Path('chips')
preprocessed_path.mkdir(exist_ok=True)

utm = planet_crs['init'].split(':')[-1]
utm_output = preprocessed_path/f'{planet_image_path.stem}_cropped_to_chip_{utm}_{index}.tif'
with rasterio.open(utm_output, 'w', **profile_rgnir_cropped) as ds:
    ds.write(rgnir_cropped.transpose([2, 0, 1]))

### Reproject preprocessed chip to latlon

In [ ]:
dst_crs = 'EPSG:4326'

source = utm_output
latlon_output = preprocessed_path/f'{planet_image_path.stem}_cropped_to_chip_{index}_latlon.tif'

with rasterio.open(source) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(latlon_output, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

### Delete original planet data and cropped chip in utm

In [ ]:
try:
    shutil.rmtree(planet_dir)
except FileNotFoundError:
    pass

try:
    utm_output.unlink()
except FileNotFoundError:
    pass

### Plot pre-processed Planet data

In [ ]:
from rasterio.plot import show
import matplotlib.pyplot as plt

def get_raster_threshold(raster, low=2, high=98):
    return [np.nanpercentile(raster, low),
            np.nanpercentile(raster, high)]

with rasterio.open(latlon_output) as ds:
    image_c = ds.read(window=window)
    image_c = image_c.transpose([1, 2, 0]).astype(np.float32)

    
    fig, (ax_r, ax_g, ax_nir) = plt.subplots(1, 3, figsize=(28,7))
    thresh = get_raster_threshold(image_c[..., 0])
    show((ds, 1), ax=ax_nir, cmap='GnBu', title='nir band', vmin=thresh[0], vmax=thresh[1])
    thresh = get_raster_threshold(image_c[..., 1])
    show((ds, 2), ax=ax_r, cmap='Reds', title='red band', vmin=thresh[0], vmax=thresh[1])
    thresh = get_raster_threshold(image_c[..., 2])
    show((ds, 3), ax=ax_g, cmap='Greens', title='green band', vmin=thresh[0], vmax=thresh[1])
print(f'Planet Image shape: {image_c.shape}')